In [ ]:
from Utils import get_embeddings, login_Hugging_for_tokenizer_model
import torch

In [ ]:
tokenizer, model = login_Hugging_for_tokenizer_model()

def smiles_to_embeddings(smiles, tokenizer=tokenizer, model=model):
    return get_embeddings(tokenizer, model, smiles)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\Yang\.cache\huggingface\token
Login successful


In [ ]:
smiles = "NCCCC"

# Get the embeddings
embedding = smiles_to_embeddings(smiles)

# Print or use the embedding as needed
print(embedding.shape)

torch.Size([1, 7, 384])


In [ ]:
from transformers import AutoModel, AutoTokenizer

def get_smiles_embedding(smiles, model_version='DeepChem/ChemBERTa-77M-MTR', max_length=128):
    model = AutoModel.from_pretrained(pretrained_model_name_or_path=model_version, output_attentions=True) # Load ChemBERT model
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_version) # Load ChemBERT tokenizer
    # Build tokenizer with same padding as in training
    tokens = tokenizer(
                smiles,
                max_length=max_length,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )

    # Get the embeddings from CLS token
    with torch.no_grad():
        emb = model(
            tokens["input_ids"],
            tokens["attention_mask"]
        )["last_hidden_state"][:, 0, :].numpy()
    
    return emb


[[-0.24266241  0.30873424  0.04592807  0.25971904 -0.2327013   0.03468586
   0.25395554 -0.09177838  0.18316105 -0.2616902  -0.2962829  -0.07542133
   0.07019384 -0.09132029 -0.08019534 -0.00945247 -0.3274298  -0.32247984
  -0.0842604   0.11031325  0.3336127  -0.00563066  0.01571712 -0.301354
  -0.24270968 -0.20475972 -0.1421593   0.23083301 -0.03602657 -0.0107397
   0.04983145  0.15676363  0.02868942  0.04773795  0.26598844  0.09794639
   0.07370745  0.09912263  0.06953901 -0.17010204  0.04149458  0.20518687
   0.06785604  0.06076902  0.0584873  -0.12422742 -0.02947365 -0.11834992
  -0.10592988 -0.01037932 -0.03798876  0.23551685 -0.07751242 -0.24929836
  -0.0463852   0.12985167 -0.22289795 -0.1335349   0.04594678 -0.28542638
  -0.27348894 -0.10488877  0.14732462 -0.09155297 -0.03746787 -0.10174252
  -0.21891917  0.20941682  0.28171122  0.06663831 -0.03003342 -0.15222865
  -0.16352792 -0.05705269 -0.27509323 -0.2041312   0.08902555 -0.1255533
   0.05932727  0.0767987  -0.10255975 -0.0